<a href="https://colab.research.google.com/github/sutoa/learn-crewai/blob/layout-aware/crewAI_agents-dependency_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is to compare client ownership structure between the ADV document and crds using crewAI agents

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 chromadb==0.4.24 "numpy<2.0" networkx==2.8.8

In [2]:
import os
# from utils import get_openai_api_key
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [3]:
!pip install --force-reinstall "numpy<2.0"


  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.


In [10]:
import numpy as np
print(np.__version__)  # should show something like 1.26.4

1.26.4


In [5]:
from crewai_tools import tool
import requests
import os

@tool("DownloadFileTool")
def download_file(url: str, destination_folder: str = ".") -> str:
    """
    Downloads a file from the given URL and saves it locally.

    Args:
        url (str): The URL of the file to download.
        destination_folder (str, optional): Where to save the file. Defaults to current directory.

    Returns:
        str: Full path to the downloaded file or an error message.
    """
    try:
        os.makedirs(destination_folder, exist_ok=True)

        # Extract base filename from URL
        original_filename = url.split("/")[-1]
        base_name, ext = os.path.splitext(original_filename)
        destination_path = os.path.join(destination_folder, original_filename)

        # Handle filename conflict by appending a number
        counter = 1
        while os.path.exists(destination_path):
            destination_path = os.path.join(destination_folder, f"{base_name}_{counter}{ext}")
            counter += 1

        # Download the file
        response = requests.get(url)
        response.raise_for_status()
        with open(destination_path, 'wb') as f:
            f.write(response.content)

        return f"File downloaded successfully: {destination_path}"
    except Exception as e:
        return f"Failed to download file from {url}. Error: {str(e)}"


In [ ]:
import requests
from crewai_tools import BaseTool
from typing import Literal

class FileDownloaderTool(BaseTool):
    name: Literal["FileDownloaderTool"] = "FileDownloaderTool"  # Add name attribute with type annotation
    description: str = "Downloads a file from a URL and saves it to a local path."  # Add description attribute with type annotation

    def _run(self, file_url: str, save_path: str):
        """Downloads a file from the given URL and saves it locally."""
        try:
            response = requests.get(file_url, stream=True)
            response.raise_for_status()  # Check for errors
            with open(save_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            return f"File downloaded successfully: {save_path}"
        except Exception as e:
            return f"Error downloading file: {str(e)}"

In [8]:
import networkx as nx
import matplotlib.pyplot as plt
from crewai import Task, Agent, Crew
from crewai_tools import SerperDevTool, PDFSearchTool

# Define agents
researcher = Agent(
    role="KYC Researcher",
    goal="Download the ADV form for {client} from government website advisorinfo.sec.gov",
    backstory="""As a KYC researcher supporting the onboarding process for client at a major European bank,
        Your responsibilities are to collect the most up-to-date information about the client, {client},
        from internal and external data sources to be analyzed by KYC analyst on the case.
        For client ownership information, you need to download the ADV form for {client} to a local folder.
        The ADV form is usually available at the sec.gov website. For instance, the download link for Goldman Sachs is https://reports.advisorinfo.sec.gov/reports/ADV/361/PDF/361.pdf.
    """,
    verbose=True,
    allow_delegation=False,
    tools=[download_file, SerperDevTool()]
    )


rag_tool = PDFSearchTool(
    top_k=5,                        # Number of search results to return
    chunk_size=600,                # Token or character length per chunk
    chunk_overlap=200,              # Overlap between chunks
    embedder="openai",             # or "huggingface", etc.
    vector_store="chromadb",       # Currently default and local
)

analyst = Agent(
     role="KYC Analyst",
     goal="Extract the ownership structure information from document or data collected by the research for client {client}",
     backstory="""As a KYC analyst for the bank, one of you jobs is to collect the list of DIRECT Owners and Excecutive Officers for client {client} from its ADV document.
     The information can be found from the Schedule A section in the form. The result should meet the following criteria:
      - if no list is found, then just return an empty list. Do NOT return any fake names.
      - include the name, title and ownership code and description for each person.
      - Note that for ownership, The description for each code can be found also in 'Schedule A'. For instance, for ownership code 'B', the description is '10% but less than 25%'. For 'C', the description is  '25% but less than 50%'. Do NOT guess or give your own ownership description.
     """,
     verbose=True,
     allow_delegation=False,
     tools=[rag_tool]
)
# reviewer = Agent(
#     role="KYC Reviewer",
#     goal="To ensure that differences in ownership structure data from various sources for the same client are reported accurately and thoroughly, listed in bullets with values that are different",
#     backstory="reviewer's job is to ensure the quality of the report by analyst, making sure they are accurate, complete and easily to read.")

# Define tasks
research_task = Task(description="Gather ADV form for {client}",
                     agent=researcher,
                     expected_output="The downloaded copy of the ADV file for client {client} on the local drive and its complete file path")
analyze_task = Task(description="Retrieve the list of Direct Owners and Executive Officers for {client} from its ADV document",
                    agent=analyst,
                    expected_output="A json file with the list of direct owners and executive officers of {client}")
# review_task = Task(description="Review delta report by analyst. if unsatisfied,  ask analyst to improve with detailed feedback. ", agent=reviewer, expected_output="feedback on the delta report")

# Create a task graph
# task_graph = nx.DiGraph()
# task_graph.add_edge(research_task, analyze_task)
# task_graph.add_edge(analyze_task, review_task)

# Visualize the task graph
# nx.draw(task_graph, with_labels=True, node_size=2000, node_color="lightblue", font_size=7)
# plt.show()

# Create crew
crew = Crew(
    agents=[researcher, analyst],
    tasks=[research_task, analyze_task],
    verbose=2
)

# Execute crew
inputs = {
    "client": "Boston Partners Global Investors, Inc.",
    #"client": "Deutche Bank Securities, Inc.",
    "inquiry": "Please download the ADV form for the client"
}
result = crew.kickoff(inputs)
print(result)

 [DEBUG]: == Working Agent: KYC Researcher
 [INFO]: == Starting Task: Gather ADV form for Deutche Bank Securities, Inc.


> Entering new CrewAgentExecutor chain...
I need to find the ADV form for Deutsche Bank Securities, Inc. on the SEC website. I will start by searching for the ADV form to find the correct URL for downloading it.

Action: Search the internet  
Action Input: {"search_query": "Deutche Bank Securities, Inc. ADV form site:sec.gov"}  
 


Search results: Title: DEUTSCHE BANK SECURITIES INC.
Link: https://adviserinfo.sec.gov/firm/summary/2525
Snippet: An ERA is required to file a report using Form ADV, but does not complete all items contained in Form ADV that a registered adviser must complete. Other ...
---
Title: FORM ADV
Link: https://reports.adviserinfo.sec.gov/reports/ADV/2525/PDF/2525.pdf
Snippet: DEUTSCHE BANK SECURITIES INC. B. (1) Name under which you primarily conduct your advisory business, if different from Item 1.A. DEUTSCHE ...
---
Title: FORM ADV
Link: http

Inserting batches in chromadb: 100%|██████████| 20/20 [00:20<00:00,  1.03s/it]


 

Relevant Content:
If you are submitting an initial application or report, you must complete Schedule A and Schedule B. Schedule A asks for information about your direct owners and executive officers. Schedule B asks for information about your indirect owners. If this is an amendment and you are updating information you reported on either Schedule A or Schedule B (or both) that you filed with your initial application or report, you must complete Schedule C. Yes No A. Does any person not named in Item 1.A. or Schedules A, B, or C, directly or indirectly, control your management or policies? If yes, complete Section 10.A. of Schedule D.

application or report. Schedule B asks for information about your indirect owners; you must first complete Schedule A, which asks for information about your direct owners. Use Schedule C to amend this information. 2.Indirect Owners. With respect to each owner listed on Schedule A (except individual owners), list below: (a)in the case of an owner that i

### Below to try layout-aware chunking for better search result

In [15]:
!pip install "pdfminer.six==20221105" unstructured sentence-transformers chromadb openai pi-heif "unstructured[local-inference]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.1/526.1 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 71.6 MB/s eta 0:00:00
  Cr

In [14]:
from crewai_tools import tool
from unstructured.partition.pdf import partition_pdf
from sentence_transformers import SentenceTransformer
from chromadb import Client
from chromadb.config import Settings
import openai
import os

@tool("LayoutAwarePDFQATool")
def layout_pdf_qa_tool(params: dict) -> str:
    """
    Answers a prompt based on a layout-aware read of a PDF using ChromaDB + LLM.

    Args (params dict):
        pdf_path (str): Path to the local PDF file.
        prompt (str): Natural language question.
        top_k (int, optional): Number of top chunks to retrieve (default: 5)

    Returns:
        str: Final LLM-generated answer.
    """
    try:
        pdf_path = params.get("pdf_path")
        prompt = params.get("prompt")
        top_k = params.get("top_k", 5)

        if not pdf_path or not prompt:
            return "❌ Error: Missing 'pdf_path' or 'prompt'."

        if not os.path.exists(pdf_path):
            return f"❌ File not found: {pdf_path}"

        # Step 1: Parse PDF layout-aware chunks
        elements = partition_pdf(filename=pdf_path)
        texts = [el.text for el in elements if el.text and len(el.text.strip()) > 20]

        if not texts:
            return "⚠️ No readable content found in the PDF."

        # Step 2: Embed chunks
        embedder = SentenceTransformer("all-MiniLM-L6-v2")
        vectors = embedder.encode(texts, convert_to_tensor=True).tolist()

        # Step 3: Create Chroma collection
        chroma_client = Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory=".chroma_pdfqa"))
        collection = chroma_client.get_or_create_collection(name="layout_pdf_qa")

        ids = [f"{os.path.basename(pdf_path)}_chunk_{i}" for i in range(len(texts))]
        collection.add(documents=texts, embeddings=vectors, ids=ids)

        # Step 4: Query top-k relevant chunks
        results = collection.query(query_texts=[prompt], n_results=top_k)
        top_chunks = results["documents"][0]

        if not top_chunks:
            return "No relevant content found."

        context = "\n\n".join(top_chunks)

        # Step 5: Use LLM to answer
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You're a PDF analyst answering using the context below."},
                {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {prompt}"}
            ],
            temperature=0.2
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"❌ Unexpected error: {str(e)}"


ModuleNotFoundError: No module named 'unstructured_inference'